In [37]:
import pandas as pd
import numpy as np

In [38]:
#Reading the train file
train_df= pd.read_csv("train.csv", parse_dates=['impression_time'] , index_col='impression_id')
# Parsing the dates and indexing the impression ids
train_df.head()

,impression_time,user_id,app_code,os_version,is_4G,is_click
impression_id,,,,,,
c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


In [39]:
final_train= train_df.drop(['app_code'], axis=1)

In [22]:
#final_train = final_train[(final_train['impression_time']>=pd.Timestamp(2018,11,15)) & 
                #            (final_train['impression_time']<pd.Timestamp(2018,12,12))]

In [40]:
#Encoding string columns
from sklearn.preprocessing import LabelEncoder

In [41]:
os_le = LabelEncoder()
final_train.loc[:,'os_label'] = os_le.fit_transform(final_train['os_version'])

In [42]:
final_input= final_train.drop(['impression_time','os_version'],axis='columns')
final_input.reset_index(inplace=True) # Removing impression id index as it might cause confusion
final_input.drop('impression_id', axis=1, inplace=True)
train_data = final_input.drop(['is_click'], axis=1)
target = final_input['is_click']

In [43]:
#Setting up a tree and training it
from sklearn import tree

In [44]:
model= tree.DecisionTreeClassifier()
model.fit(train_data,target)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [45]:
#Creating test dataframe
#Reading the train file
test_data= pd.read_csv("test.csv", parse_dates=['impression_time'])

In [46]:
test_data.drop(['impression_id', 'impression_time', 'app_code'], axis=1, inplace=True)

In [47]:
osle = LabelEncoder()
test_data.loc[:,'os_label'] = osle.fit_transform(test_data['os_version'])

In [48]:
test_data.drop('os_version', axis=1, inplace=True)

In [49]:
answer = model.predict(test_data)

In [50]:
#Creating submission file using dataframe
submit_data= pd.read_csv("test.csv")
submit_data.drop(['impression_time','user_id','app_code','os_version','is_4G'], axis=1, inplace=True)
submit_data.to_csv('submission.csv', index=False)

In [51]:
#Converting into np array
predict_array = np.array(answer)
np.shape(predict_array)

(90675,)

In [52]:
#Writing the predicted array to another file in a column
import csv
with open('prediction.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for row in range(0,predict_array.shape[0]):
        myList = []
        myList.append(predict_array[row])
        writer.writerow(myList)

In [53]:
# Now bringing them together
a = pd.read_csv("submission.csv")
b = pd.read_csv("prediction.csv", header=None, names=["is_click"])
final_submission = pd.concat([a,b],axis=1)
final_submission.to_csv('final_submission.csv', index=False)

#The End